# Visitare il grafo (implicito) della distanza di Levenshtein (soluzione)

La [distanza di Levenshtein](https://www.wikiwand.com/en/Levenshtein_distance), che denoteremo con $d_L(u, v)$ può essere calcolata tramite una funzione ricorsiva

In [ ]:
def d_L(u, v):
  if not u: return len(v)
  if not v: return len(u)
  if u[0] == v[0]: return d_L(u[1:], v[1:])
  return 1 + min(
    d_L(u[1:], v),
    d_L(u, v[1:]),
    d_L(u[1:], v[1:])
  )

In [ ]:
# esempio 

d_L('CANE', 'CASA')

2

Otteniamo ora l'elenco di parole che ci servirà per definire il grafo.

In [ ]:
from urllib.request import urlopen

with urlopen('https://raw.githubusercontent.com/napolux/paroleitaliane/master/paroleitaliane/60000_parole_italiane.txt') as url: 
  W = frozenset({word.decode().strip().upper() for word in url if len(word) >= 3})

print(len(W))

60418


Diamo una prima implementazione (inefficiente) della funzione $\Gamma_d$ (che implementeremo con una funzione a due parametri, il primo dei quali svolge il ruolo di $d$)


In [ ]:
def Γ(d, u):
  return frozenset(w for w in W if d_L(u, w) <= d)

In [ ]:
# cacoliamo Γ_1('CASA'), prendendo nota del tempo di calcolo

%time ineff = Γ(1, 'CASA')

CPU times: user 30.9 s, sys: 2.51 ms, total: 30.9 s
Wall time: 30.9 s


Ora diamo una implementazione (efficiente) per il caso $d = 1$

In [ ]:
from string import ascii_uppercase

def Γ_1(u):
  res = set()
  for i in range(len(u) + 1):
    res.add(u[:i] + u[i+1:]) # cancella un carattere
    for c in ascii_uppercase:
      res.add(u[:i] + c + u[i+1:]) # sostituisce un carattere
      res.add(u[:i] + c + u[i:]) # inserisce un carattere
  return res & W

In [ ]:
# rifacciamo il calcolo, sempre annotando il tempo 

%time eff = Γ_1('CASA')

CPU times: user 72 µs, sys: 0 ns, total: 72 µs
Wall time: 73.2 µs


In [ ]:
# i due risultati coincidono (ovviamente)

ineff == eff

True

Usando `Γ_1` scriviamo l'algoritmo di visita che ci permetta di ricostruire il percorso tra due parole date. 

Una possibilità è quella di usare una struttura dati ausiliaria che ci consenta di tener traccia, per ciascuna parola, della parola che l'ha preceduta nella visita. 

Non ponendosi un problema di efficienza, una scelta senz'altro più banale da implementare è quella di tenere nella coda non le parole, ma gli interi percorsi.

In [ ]:
from liblet import Queue

def visit(start, stop):
  Q = Queue([(start, )]) # all'inizio il percorso è solo start
  seen = set()
  while Q:
    path = Q.dequeue()
    u = path[-1] # la parola da cui procedere è l'ultima del pecorso
    seen.add(u)
    if u == stop: return path
    for w in Γ_1(u):
      if not w in seen:
        # il percorso esteso è ottenuto accodando la parola adiacente 
        augmented_path = path +(w, )
        Q.enqueue(augmented_path) 
  return None

In [ ]:
visit('CASA', 'MIA')

('CASA', 'COSA', 'OSA', 'SA', 'SIA', 'MIA')